-reference
- https://github.com/lovit/clustering4docs
- https://programmers.co.kr/learn/courses/21/lessons/1700
- https://lovit.github.io/nlp/machine%20learning/2018/10/16/spherical_kmeans/
- https://lovit.github.io/nlp/machine%20learning/2018/03/21/kmeans_cluster_labeling/
- https://github.com/lovit/kmeans_to_pyLDAvis/blob/master/jupyter/visualize_kmeans_using_pyLDAvis.ipynb

In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('영어 크롤링.csv',  encoding='cp1252')
#보통은 utf8을 인코더로 씀.

In [3]:
data

,Date,Name,Score,Review,like
0,26-Jan-20,Bernard Reis,1,Completely frustrating experience. Paid extra ...,7
1,18-Jan-20,Gordon Inman,2,"Some good features, but this app isn't very sm...",8
2,18-Jan-20,MrDurma,4,"This app works perfectly, it allowed us to dow...",2
3,26-Jan-20,K Kostas,3,Good but not great for an app by LG. Not the m...,1
4,17-Jan-20,Greg Evans,3,App is useful but limited and requires some up...,1
...,...,...,...,...,...
7145,31-Dec-19,Mr Darryl Maurice Friend,5,Wonderful,0
7146,12-Jan-20,luis arias,5,Wonderful,0
7147,24-Sep-17,Tobiloba Bakare,5,Awesome ?? ?? ?? ??,0
7148,13-Oct-17,A Google user,5,Good Good Good,0


# 전처리

In [4]:
#중복값이 있다면 첫번째 데이터 삭제, 중복 데이터 확인
data.duplicated(['Review'], keep='first')
data.Review.is_unique

False

In [5]:
# abstract_list=[]
# for i in list(data['abstract']):
#     a = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', ' ', i)
#     b = re.sub('[1-9]', "", i)#숫자 제거
#     space_remove=' '.join(a.split())# 다중 공백 제거
#     abstract_list.append(space_remove.lower()) # 소문자로 변환 후 append

In [6]:
review = data['Review'].tolist()
review ='.'.join(review)
review

'Completely frustrating experience. Paid extra money to get the air conditioner with connectivity, but it just doesn\'t work 9/10 times I try to use it. When it gets to connect to the device, it takes less than a minute to get disconnected again. Tried reinstalling the app, cleared the cache, reset router, replaced the router, but nothing works. Again, I\'m completely frustrated..Some good features, but this app isn\'t very smart. You would think once you connect to the device it would give the UI/UX for your specific device. I connected my new stove/oven, but the image used to represent my stove doesn\'t match my stovetop burner layout. The directions for the oven remote start are written generically to try to fit any product; "look to see if you have this." If they would put some effort into personalizing the experience to the device you purchased, I\'d rate this higher..This app works perfectly, it allowed us to download many more cycles for our Washer/Dryer. It really has cool feat

In [7]:
review_list=[]
for i in list(data['Review']):
    a = re.sub('[^a-zA-Z]', ' ', i) #알파벳만,
    space_remove=' '.join(a.split())# 다중 공백 제거
    review_list.append(space_remove.lower()) # 소문자로 변환 후 append

In [8]:
review_list

['completely frustrating experience paid extra money to get the air conditioner with connectivity but it just doesn t work times i try to use it when it gets to connect to the device it takes less than a minute to get disconnected again tried reinstalling the app cleared the cache reset router replaced the router but nothing works again i m completely frustrated',
 'some good features but this app isn t very smart you would think once you connect to the device it would give the ui ux for your specific device i connected my new stove oven but the image used to represent my stove doesn t match my stovetop burner layout the directions for the oven remote start are written generically to try to fit any product look to see if you have this if they would put some effort into personalizing the experience to the device you purchased i d rate this higher',
 'this app works perfectly it allowed us to download many more cycles for our washer dryer it really has cool features and we don t have any

## 토큰화 / 불용어제거

In [9]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords

In [10]:
stop_words=set(stopwords.words('english'))

In [11]:
result=[]

for line in review_list:
    tokens = word_tokenize(line)
    stopped_tokens = [i for i in list(tokens) if not i in stop_words]
    stopped_tokens2 = [i for i in stopped_tokens if len(i)>1] #한글자 이상인 토큰들
    result.append(stopped_tokens2)
                      

In [12]:
result

[['completely',
  'frustrating',
  'experience',
  'paid',
  'extra',
  'money',
  'get',
  'air',
  'conditioner',
  'connectivity',
  'work',
  'times',
  'try',
  'use',
  'gets',
  'connect',
  'device',
  'takes',
  'less',
  'minute',
  'get',
  'disconnected',
  'tried',
  'reinstalling',
  'app',
  'cleared',
  'cache',
  'reset',
  'router',
  'replaced',
  'router',
  'nothing',
  'works',
  'completely',
  'frustrated'],
 ['good',
  'features',
  'app',
  'smart',
  'would',
  'think',
  'connect',
  'device',
  'would',
  'give',
  'ui',
  'ux',
  'specific',
  'device',
  'connected',
  'new',
  'stove',
  'oven',
  'image',
  'used',
  'represent',
  'stove',
  'match',
  'stovetop',
  'burner',
  'layout',
  'directions',
  'oven',
  'remote',
  'start',
  'written',
  'generically',
  'try',
  'fit',
  'product',
  'look',
  'see',
  'would',
  'put',
  'effort',
  'personalizing',
  'experience',
  'device',
  'purchased',
  'rate',
  'higher'],
 ['app',
  'works',
  '

### 1. stemming

In [13]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
s = PorterStemmer()

In [14]:
print([s.stem(w) for w in result[0]])

['complet', 'frustrat', 'experi', 'paid', 'extra', 'money', 'get', 'air', 'condition', 'connect', 'work', 'time', 'tri', 'use', 'get', 'connect', 'devic', 'take', 'less', 'minut', 'get', 'disconnect', 'tri', 'reinstal', 'app', 'clear', 'cach', 'reset', 'router', 'replac', 'router', 'noth', 'work', 'complet', 'frustrat']


In [15]:
stemming_data=[]
for i in result:
    stemming_data.append([s.stem(w) for w in i])

In [16]:
stemming_data

[['complet',
  'frustrat',
  'experi',
  'paid',
  'extra',
  'money',
  'get',
  'air',
  'condition',
  'connect',
  'work',
  'time',
  'tri',
  'use',
  'get',
  'connect',
  'devic',
  'take',
  'less',
  'minut',
  'get',
  'disconnect',
  'tri',
  'reinstal',
  'app',
  'clear',
  'cach',
  'reset',
  'router',
  'replac',
  'router',
  'noth',
  'work',
  'complet',
  'frustrat'],
 ['good',
  'featur',
  'app',
  'smart',
  'would',
  'think',
  'connect',
  'devic',
  'would',
  'give',
  'ui',
  'ux',
  'specif',
  'devic',
  'connect',
  'new',
  'stove',
  'oven',
  'imag',
  'use',
  'repres',
  'stove',
  'match',
  'stovetop',
  'burner',
  'layout',
  'direct',
  'oven',
  'remot',
  'start',
  'written',
  'gener',
  'tri',
  'fit',
  'product',
  'look',
  'see',
  'would',
  'put',
  'effort',
  'person',
  'experi',
  'devic',
  'purchas',
  'rate',
  'higher'],
 ['app',
  'work',
  'perfectli',
  'allow',
  'us',
  'download',
  'mani',
  'cycl',
  'washer',
  'dry

### 2. Lemmatization

In [17]:
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

lemmatize_data=[]
for i in result:
    lemmatize_data.append([lemmatizer.lemmatize(w) for w in i])
    
lemmatize_data

[['completely',
  'frustrating',
  'experience',
  'paid',
  'extra',
  'money',
  'get',
  'air',
  'conditioner',
  'connectivity',
  'work',
  'time',
  'try',
  'use',
  'get',
  'connect',
  'device',
  'take',
  'le',
  'minute',
  'get',
  'disconnected',
  'tried',
  'reinstalling',
  'app',
  'cleared',
  'cache',
  'reset',
  'router',
  'replaced',
  'router',
  'nothing',
  'work',
  'completely',
  'frustrated'],
 ['good',
  'feature',
  'app',
  'smart',
  'would',
  'think',
  'connect',
  'device',
  'would',
  'give',
  'ui',
  'ux',
  'specific',
  'device',
  'connected',
  'new',
  'stove',
  'oven',
  'image',
  'used',
  'represent',
  'stove',
  'match',
  'stovetop',
  'burner',
  'layout',
  'direction',
  'oven',
  'remote',
  'start',
  'written',
  'generically',
  'try',
  'fit',
  'product',
  'look',
  'see',
  'would',
  'put',
  'effort',
  'personalizing',
  'experience',
  'device',
  'purchased',
  'rate',
  'higher'],
 ['app',
  'work',
  'perfectly

In [18]:
print([lemmatizer.lemmatize(w) for w in result[0]])

['completely', 'frustrating', 'experience', 'paid', 'extra', 'money', 'get', 'air', 'conditioner', 'connectivity', 'work', 'time', 'try', 'use', 'get', 'connect', 'device', 'take', 'le', 'minute', 'get', 'disconnected', 'tried', 'reinstalling', 'app', 'cleared', 'cache', 'reset', 'router', 'replaced', 'router', 'nothing', 'work', 'completely', 'frustrated']


## 포스태깅

In [19]:
import nltk
# nltk.download('all')

In [20]:
nltk.pos_tag(lemmatize_data[3])

[('good', 'JJ'),
 ('great', 'JJ'),
 ('app', 'NN'),
 ('lg', 'NN'),
 ('user', 'JJ'),
 ('interface', 'NN'),
 ('friendly', 'JJ'),
 ('adjustment', 'NN'),
 ('user', 'NN'),
 ('downloaded', 'VBD'),
 ('add', 'RP'),
 ('washer', 'PRP$'),
 ('dryer', 'JJ'),
 ('extra', 'JJ'),
 ('cycle', 'NN'),
 ('cycle', 'NN'),
 ('would', 'MD'),
 ('nice', 'VB'),
 ('like', 'IN'),
 ('soak', 'JJ'),
 ('cycle', 'NN'),
 ('bit', 'NN'),
 ('different', 'JJ'),
 ('cycle', 'NN'),
 ('adjusting', 'VBG'),
 ('temp', 'JJ'),
 ('existing', 'VBG'),
 ('one', 'CD')]

* pos-tagging 리스트 넣기

In [21]:
pos_tagged=[]
for i in lemmatize_data:
    pos_tagged.append(nltk.pos_tag(i))

In [22]:
pos_tagged

[[('completely', 'RB'),
  ('frustrating', 'JJ'),
  ('experience', 'NN'),
  ('paid', 'VBD'),
  ('extra', 'JJ'),
  ('money', 'NN'),
  ('get', 'VB'),
  ('air', 'NN'),
  ('conditioner', 'NN'),
  ('connectivity', 'NN'),
  ('work', 'NN'),
  ('time', 'NN'),
  ('try', 'VB'),
  ('use', 'NN'),
  ('get', 'VB'),
  ('connect', 'JJ'),
  ('device', 'NNS'),
  ('take', 'VBP'),
  ('le', 'JJR'),
  ('minute', 'NN'),
  ('get', 'NN'),
  ('disconnected', 'VBN'),
  ('tried', 'JJ'),
  ('reinstalling', 'VBG'),
  ('app', 'NN'),
  ('cleared', 'VBD'),
  ('cache', 'NN'),
  ('reset', 'NN'),
  ('router', 'NN'),
  ('replaced', 'VBD'),
  ('router', 'NN'),
  ('nothing', 'NN'),
  ('work', 'NN'),
  ('completely', 'RB'),
  ('frustrated', 'VBD')],
 [('good', 'JJ'),
  ('feature', 'NN'),
  ('app', 'IN'),
  ('smart', 'NN'),
  ('would', 'MD'),
  ('think', 'VB'),
  ('connect', 'JJ'),
  ('device', 'NN'),
  ('would', 'MD'),
  ('give', 'VB'),
  ('ui', 'JJ'),
  ('ux', 'JJ'),
  ('specific', 'JJ'),
  ('device', 'NN'),
  ('connected', 

* pos-tagging 품사 선택 부분

In [23]:
selected_pos_tagged_voca=[]
for i in pos_tagged:
    selected_pos_tagged_voca.append([word for word, pos in i if pos in ['NN', 'NNP', 'NNS', 'NNPS', 'VBG']])

In [24]:
selected_pos_tagged_voca

[['experience',
  'money',
  'air',
  'conditioner',
  'connectivity',
  'work',
  'time',
  'use',
  'device',
  'minute',
  'get',
  'reinstalling',
  'app',
  'cache',
  'reset',
  'router',
  'router',
  'nothing',
  'work'],
 ['feature',
  'smart',
  'device',
  'device',
  'stove',
  'image',
  'match',
  'stovetop',
  'burner',
  'direction',
  'start',
  'product',
  'look',
  'effort',
  'personalizing',
  'experience',
  'device',
  'rate',
  'higher'],
 ['work',
  'cycle',
  'dryer',
  'feature',
  'problem',
  'thing',
  'kind',
  'bothering',
  'power',
  'monitoring',
  'section',
  'show',
  'electricity',
  'show',
  'rating',
  'kwh',
  'kwh',
  'rating',
  'experience'],
 ['app',
  'lg',
  'interface',
  'adjustment',
  'user',
  'cycle',
  'cycle',
  'cycle',
  'bit',
  'cycle',
  'adjusting',
  'existing'],
 ['setup',
  'button',
  'reference',
  'user',
  'turn',
  'wifi',
  'using',
  'use',
  'analysis',
  'function',
  'test',
  'pas',
  'problem',
  'getting',


## Word2Vec

In [25]:
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import word2vec

 - sg=1 : skipgram / sg=0 : cbow
 - size : 임베딩된 벡터의 차원
 - window : 중심단어로부터 좌우 몇개의 단어까지 적용할 것인지
 - min_count : 전체 문서에서 최소 몇번 이상 출현한 단어들을 대상으로 학습 할 것인지
 - workers : 학습을 위하나 프로세스 수 

In [26]:
#기본적으로 코사인 유사도를 기반 모델만들기
embedding_model = Word2Vec(selected_pos_tagged_voca, 
                           sg=1, window=2, min_count=3)
#sg =0은 CBOW / 1은 skip-gram
#window = 좌우2개 단어까지 학습에 적용
#min_count= 최소 3개까지 나온 단어를 학습대상으로 놓음

In [27]:
#gensim에서는 기본 유사도를 코사인 유사도를 이용
embedding_model.most_similar("information")

[('gps', 0.9994352459907532),
 ('giving', 0.9994016885757446),
 ('grant', 0.9993610382080078),
 ('requirement', 0.9993499517440796),
 ('data', 0.9993399381637573),
 ('language', 0.9993367195129395),
 ('asking', 0.999306321144104),
 ('requiring', 0.9993022680282593),
 ('claim', 0.9993017911911011),
 ('camera', 0.9992848634719849)]

In [28]:
embedding_model.similarity("information", "connection")

0.9963411

In [29]:
#모델 저장
embedding_model.save('vectorization_ieee')

In [30]:
#모델 불러오기
#  model = Word2Vec.load(fname)

model=Word2Vec.load('vectorization_ieee')

# Doc2Vec

In [31]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

In [32]:
for i in range(len(selected_pos_tagged_voca)):
    selected_pos_tagged_voca[i] = TaggedDocument(words = selected_pos_tagged_voca[i], 
                                                 tags = ['sent{}'.format(i)])    
    # converting each sentence into a TaggedDocument
    # tags = sent(문장번호)로 놓겠다. --> 보통 감정으로 넣음.pos/neg

In [41]:
selected_pos_tagged_voca[2]

TaggedDocument(words=['work', 'cycle', 'dryer', 'feature', 'problem', 'thing', 'kind', 'bothering', 'power', 'monitoring', 'section', 'show', 'electricity', 'show', 'rating', 'kwh', 'kwh', 'rating', 'experience'], tags=['sent2'])

In [42]:
doc_model = Doc2Vec(
    documents = selected_pos_tagged_voca, 
    dm = 1, size = 100, window = 3, min_count = 1)
#dm = 1은 PV-DM / 0은 PV-DBOW
#size = vector사이즈
#window 앞뒤 3단어씩 학습시킨다.
#min_count = 1개 이상의 단어만 학습시킨다.
doc_model.init_sims(replace = True)

In [43]:
print(doc_model.wv.most_similar('air'))

[('network', 0.9997223615646362), ('product', 0.9997129440307617), ('device', 0.9996961951255798), ('control', 0.9996898174285889), ('functionality', 0.9996881484985352), ('system', 0.999687910079956), ('connection', 0.9996840953826904), ('show', 0.9996802806854248), ('unit', 0.9996800422668457), ('running', 0.9996798038482666)]


In [44]:
model_name = 'Doc2vec.model' # 모델 이름 'Doc2vec.moel'로 설정
doc_model.save(model_name) # doc2vec 모델 저장

In [45]:
# doc_vectorizer라는 이름으로 저장한 모델 불러옴
doc_model = Doc2Vec.load(model_name)